# Project Five 
# Game of Thrones - Season Five

## Importing and Cleaning

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import gensim
from nltk.tag import pos_tag
import re

Using Theano backend.


In [2]:
pd.set_option('display.max_columns', 125)
pd.set_option('display.max_rows', 125)

In [3]:
df_s5 = pd.read_json('/Users/zoemarkovits/Metis/project_five_got/scrape/season_five.json')

In [4]:
df_s5[['title','script_text']]

,title,script_text
0,[The Wars to Come Script],"[\n, \n, EXT. WOODS, \n, \n, \n, \nMELARA: We ..."
1,[The House of Black and White],"[\n, \nBRAAVOSI MAN: In the old times, wheneve..."
2,[High Sparrow Script],"[TITLE SEQUENCE, \n, \n, CUT TO: BRAAVOS, \n, ..."
3,[Sons of the Harpy Script],"[TITLE SEQUENCE, \n, \n, EXT. BEACH, \n, \n, \..."
4,[Kill the Boy Script],"[TITLE SEQUENCE, \n, \n, INT. ROOM, \n, \n, \n..."
5,"[Unbowed, Unbent, Unbroken Script]","[INT. HOUSE OF BLACK AND WHITE - CHAMBER, \n, ..."
6,[The Gift Script],"[EXT. CASTLE BLACK, \n, \n, \n, \nJON: First R..."
7,[Hardhome Script],"[INT. MEEREEN - PYRAMID, \n, \nJORAH: Your Gra..."
8,[The Dance of Dragons (Script)],"[EXT. STANNIS’ CAMP - NIGHT, \n, \n, \n, \n, I..."
9,[Mother's Mercy Script],"[INT. STANNIS’S CAMP - MELISANDRE’S TENT, \n, ..."


In [5]:
df_s5['title'] = df_s5['title'].map(lambda x: str(x)[3:-2])

In [6]:
df_s5.title

0             The Wars to Come Script
1        The House of Black and White
2                 High Sparrow Script
3            Sons of the Harpy Script
4                 Kill the Boy Script
5    Unbowed, Unbent, Unbroken Script
6                     The Gift Script
7                     Hardhome Script
8       The Dance of Dragons (Script)
9               Mother's Mercy Script
Name: title, dtype: object

In [7]:
names5_01 = ['MELARA','CERSEI','MAGGY','PRIEST','JAIME','VARYS','TYRION','PROSTITUTE','WHITE RAT','DAENERYS',
             'BARRISTAN','MOSSADOR','GREY WORM','MISSANDEI','JON','OLLY','GILLY','SAM','ALLISER','MELISANDRE',
             'STANNIS','DAVOS','MASTER OF ARMS','YOHN ROYCE','BAELISH','ROYCE','SQUIRE','SANSA','PODRICK','BRIENNE',
             'LORAS','GRAND MAESTER PYRCELLE','LANCEL LANNISTER','LANCEL','KEVAN LANNISTER','KEVAN','OLYVAR',
             'MARGAERY','DAARIO NAHARIS','HIZDAHR ZO LORAQ','HIZDAHR','DAARIO','MANCE']

names5_02 = ['BRAAVOSI MAN','ARYA','MAN','OLD MAN','PODRICK','WAITRESS','SANSA','BAELISH','BRIENNE','GUARD','KNIGHT 1',
             'KNIGHT','JAIME','CERSEI','LOLLYS STOKEWORTH','BRONN','LOLLYS','TRYSTANE (O.S.)','AREO','ELLARIA',
             'DORAN','DAARIO (O.S.)','DAARIO','GREY WORM','MOSSADOR','DAENERYS','BARRISTAN','HIZDAHR','MOSSADOR',
             'TYRION','VARYS','MERYN','HUNTERS','QYBURN','KEVAN','MACE TYRELL','PYCELLE','MACE','SHIREEN','GILLY',
             'SAM','SELYSE (to Sam and Gilly):','SAM','SELYSE','STANNIS','JON','DAVOS','ALLISER','AEMON','JANOS',
             'MEN','SHADOW TOWER BROTHER','BROTHER','BROTHERS','JAQEN','HARPY (Valyrian)']

names5_03 = ['MAN','JAQEN','ARYA','MARGAERY & TOMMEN','TOMMEN','MARGAERY','CERSEI','ROOSE','RAMSAY','SANSA','BAELISH',
             'PODRICK','BRIENNE','STANNIS','JON','DAVOS','OLLY','WAIF','ROOSE','RAMSAY','WOMAN','SAM','JANOS',
             'OLYVAR','HIGH SEPTON','LANCEL','CROWD','MAN 1','MAN 2','QYBURN','MACE','PYCELLE','MERYN','MAN 3',
             'WOMAN','WOMAN 2','TYRION','VARYS','PRIESTESS','JORAH']

names5_04 = ['JAIME','CAPTAIN','BRONN','CERSEI','MACE','PYCELLE','HIGH SPARROW','MERCHANT','OLYVAR','MILITANT',
             'LORAS','LANCEL','TOMMEN','MARGAERY','KINGSGUARD','MAN 1','MAN 2','MAN 3','WOMAN','MAN 4','SELYSE','JON',
             'STANNIS','MELISANDRE','SAM','SHIREEN','SANSA','BAELISH','RIDER','TYENE','ELARIA','OBARA','NYMERIA',
             'TYRION','JORAH','DAENERYS','BARRISTAN','DARIO','HIZDAHR','MISSANDEI','WOMAN']

names5_05 = ['HIZDAHR','DAENERYS','DAARIO','MAN','SAM','AEMON','JON','TORMUND','STANNIS','DAVOS','ALLISTER','EDD',
             'OLLY','PODRICK','BRIENNE','RAMSAY','MYRANDA','WOMAN','SANSA','THEON','ROOSE','WALDA','GILLY','SHIREEN',
             'MISSANDEI','GREY WORM','TYRION','JORAH']

names5_06 = ['ARYA','WAIF','JAQEN','TYRION','JORAH','MAN','GIRL','MALKO','SLAVER','LANCEL','BAELISH','CERSEI',
             'TRYSTANE','MYRCELLA','DORAN,','AREO','JAIME','BRONN','ELLARIA','SAND SNAKES','OBARA','RIDER','OLENNA',
             'MARGAERY','HIGH SPARROW','LORAS','SPARROW','OLYVAR','MYRANDA','SANSA','THEON','ROOSE','RAMSAY']

names5_07 = ['JON','ALLISER','SAM','AEMON','SANSA','THEON','RAMSAY','AEMON','GILLY','BROTHERS','DAVOS','STANNIS',
             'MELISANDRE','DERRYK','BRAND','MALKO','SLAVE BUYER','TYRION','BUYER','DAARIO','DAENERYS','OLENNA',
             'HIGH SPARROW','SPARROW','SQUIRE','CERSEI','TOMMEN','AREO','MYRCELLA','JAIME','BRONN','TYENE','OBARA',
             'BAELISH','OLENNA','SLAVE OWNER','OWNER','MEN','HIZDAHR','JORAH','MARGAERY','HIGH SPARROW','SPARROW']

names5_08 = ['JORAH','DAENERYS','TYRION','SEPTA','CERSEI','ARYA','JAQEN','THIN MAN','MAN','WAIF','QYBURN','SANSA',
             'THEON','BOLTON BANNERMAN','ROOSE','RAMSAY','MASTER','GILLY','SAM','OLLY','TORMUND','JON',
             'LORD OF BONES','KARSI','LOBODA','WILDLING ELDER','GIANT','DAUGHTER','WUN WUN','NIGHT’S WATCH BROTHER',
             'BROTHER','EDD']

names5_09 = ['DAVOS','STANNIS','ALLISER','SAM','JON','SHIREEN','JAIME','DORAN','MYRCELLA','TRYSTANE','ELLARIA',
             'TYENE','NYMERIA','BRONN','AREO','ARYA','MAN','THIN MAN','BANKER','MACE','MERYN','BROTHEL KEEPER',
             'PROSTITUTE','JOHN','HEAD PROSTITUTE','HEAD','JAQEN','MELISANDRE','SELYSE','DAENERYS','HIZDAHR',
             'ANNOUNCER','QUICK','STRONG','DAARIO','TYRION','JORAH']

names5_10 = ['MELISANDRE','STANNIS','SOLDIER','SOLDIER 2','JON','SAM','PODRICK','BRIENNE','RAMSAY','SURVIVOR',
             'MYRANDA','THEON','SANSA','MAN','MERYN','ARYA','JAQEN','WAIF','DORAN','JAIME','ELLARIA','BRONN','TYENE',
             'MYRCELLA','TYRION','JORAH','DAARIO','GREY WORM','MISSANDEI','VARYS','DAENERYS','SEPTA','CERSEI',
             'HIGH SPARROW','SPARROW','CROWD','WOMAN','MAN','QYBURN','DAVOS','OLLY','ALLISER','BROTHER']

### Episode One

In [8]:
#df_s5.script_text[0]

In [9]:
script_text5_01 = df_s5.script_text[0]
print type(script_text5_01)

#script_text5_01

<type 'list'>


In [10]:
script_split = [i.split('\n') for i in script_text5_01]
#script_split

In [11]:
string_script_text = ' '

for i in script_split:
    for e in i:
        string_script_text = string_script_text + ' ' + e

In [12]:
#string_script_text

In [13]:
words = word_tokenize(string_script_text)
#words

In [14]:
sentences = sent_tokenize(string_script_text)
#sentences

In [15]:
dialogue = []

for sentence in sentences:
    if sentence.split(':')[0] in (names5_01):
        dialogue.append(sentence)

In [16]:
#dialogue

In [17]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s501.pkl', 'wb') as f:
    pickle.dump(dialogue, f)

In [18]:
from collections import defaultdict

speakers = defaultdict(list)
for sentence in dialogue:
    speakers[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers

In [19]:
speakers['CERSEI']

[u' Why not?',
 u' You don\u2019t need to be afraid of my father.',
 u' Yes.',
 u' Of course we should.',
 u' They said that you were terrifying.',
 u' I know you\u2019re a witch and you can see the future.',
 u'   Tell me my future or I\u2019ll have your two boring eyes gouged out of your head.',
 u' But I will be queen?',
 u' They will keep waiting.',
 u' They?',
 u' They had nothing to do with it.',
 u' ...and now our father is dead.',
 u' Tyrion may be a monster but at least he killed our father on purpose.',
 u' Cousin Lancel.',
 u' Uncle, it\u2019s quite alright.',
 u' I\u2019m sure he\u2019ll grow out of it.',
 u' Your wounds from the Blackwater seem to have healed.',
 u' What can I do for you?',
 u' What could you possibly have done to warrant my forgiveness?',
 u' I doubt you\u2019ve ever led anyone anywhere.',
 u' I don\u2019t know what you\u2019re talking about.',
 u' The day Tywin Lannister\u2019s soul needs your help\u2026         CUT TO']

In [20]:
episode_one = pd.DataFrame.from_dict(speakers, orient="index")
print episode_one.shape

(39, 22)


In [21]:
#from numpy import nan
#episode_one.fillna(value=str(0), inplace=True)

In [22]:
#episode_one.info()

In [23]:
#episode_one.head()

In [24]:
def remove_stops(sentence):
    characters = ['?','!','.',',',':',';']
    stoplist = stopwords.words('english') + characters
    try: 
        new_sentence = re.sub(r'[^a-zA-Z ]','', sentence)
        new_sentence = new_sentence.lower()
        new_sentence = new_sentence.split()
        new_sentence = [word for word in new_sentence if word not in stoplist]
        new_sentence = ' '.join(new_sentence)
        return new_sentence
    except:
        return sentence   

In [25]:
test = episode_one[0].apply(remove_stops)
#test

In [26]:
for column in episode_one.columns:
    for x in range(len(episode_one[column])):
        episode_one[column][x] = remove_stops(episode_one[column][x])

In [27]:
#episode_one.info()

In [28]:
episode_one.index = episode_one.index.str.title()

episode_one.shape

(39, 22)

In [29]:
episode_one.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Stannis,know rules winterfell,traitor plunged dagger robb starks heart,matter,swear follow ill pardon,ill offer lives freedom mance kneels swears lo...,admire dont,likes,nightfall,kneel live,None,None,None,None,None,None,None,None,None,None,None,None,None
Davos,ive talking sworn brothers,many dont,nights watch feel much affection wildlings,fair offer,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Kevan,call sparrows,course,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Melisandre,king wants word,never,feel,man woman young old lord peasant choices,free folk one true king name stannis,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Olly,heavy,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Two

In [30]:
#df_s5.script_text[1]

In [31]:
script_text5_02 = df_s5.script_text[1]
print type(script_text5_02)

#script_text5_02

<type 'list'>


In [32]:
script_split2 = [i.split('\n') for i in script_text5_02]
#script_split2

In [33]:
string_script_text2 = ' '

for i in script_split2:
    for e in i:
        string_script_text2 = string_script_text2 + ' ' + e

In [34]:
#string_script_text2

In [35]:
words2 = word_tokenize(string_script_text2)
#words2

In [36]:
sentences2 = sent_tokenize(string_script_text2)
#sentences2

In [37]:
dialogue2 = []

for sentence in sentences2:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names5_02):
            dialogue2.append(sentence)

In [38]:
#dialogue2

In [39]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s502.pkl', 'wb') as f:
    pickle.dump(dialogue2, f)

In [40]:
from collections import defaultdict

speakers2 = defaultdict(list)
for sentence in dialogue2:
    speakers2[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers2

In [41]:
speakers2['ARYA']

[u" He's just a statue.",
 u' Thank you for bringing me.',
 u' Valar dohaeris.',
 u' Hello.',
 u' Jaqen H\u2019ghar gave me this.',
 u' Please.',
 u' But wait!',
 u" You said there was no Jaqen H'ghar here.",
 u' Well who are you then?']

In [42]:
episode_two = pd.DataFrame.from_dict(speakers2, orient="index")
print episode_two.shape

(47, 23)


In [43]:
#from numpy import nan
#episode_two.fillna(value=str(0), inplace=True)

In [44]:
#episode_two.info()

In [45]:
#episode_two.head()

In [46]:
test2 = episode_two[0].apply(remove_stops)
#test2

In [47]:
for column in episode_two.columns:
    for x in range(len(episode_two[column])):
        episode_two[column][x] = remove_stops(episode_two[column][x])

In [48]:
#episode_two.info()

In [49]:
episode_two.index = episode_two.index.str.title()

episode_two.shape

(47, 23)

In [50]:
episode_two.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
Stannis,,jon,lady bear island child,know,bravery made look weak,give north,kneel,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Davos,tonight nights watch elect new lord commander,unpleasant man,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Shireen,right good,youll learn promise,three,lot time practice,greyscale,dont remember,mother,,names gilly,gilly wouldnt,None,None,None,None,None,None,None,None,None,None,None,None,None
Meryn,trying deceive queen,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Pycelle,grace past several occasions grand maester ser...,grace,must say cannot see cersei cutting,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Three

In [51]:
#df_s5.script_text[2]

In [52]:
script_text5_03 = df_s5.script_text[2]
print type(script_text5_03)

#script_text5_03

<type 'list'>


In [53]:
script_split3 = [i.split('\n') for i in script_text5_03]
#script_split3

In [54]:
string_script_text3 = ' '

for i in script_split3:
    for e in i:
        string_script_text3 = string_script_text3 + ' ' + e

In [55]:
#string_script_text3

In [56]:
words3 = word_tokenize(string_script_text3)
#words3

In [57]:
sentences3 = sent_tokenize(string_script_text3)
#sentences3

In [58]:
dialogue3 = []

for sentence in sentences3:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names5_03):
            dialogue3.append(sentence)

In [59]:
#dialogue3

In [60]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s503.pkl', 'wb') as f:
    pickle.dump(dialogue3, f)

In [61]:
from collections import defaultdict

speakers3 = defaultdict(list)
for sentence in dialogue3:
    speakers3[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers3

In [62]:
speakers3['ARYA']

[u" I've been sweeping this floor for days.",
 u" I didn't come here to sweep floors.",
 u' You said that I could be your apprentice.',
 u' I want to serve.',
 u' Which one is the Many-Faced God?',
 u' Where are they taking him?',
 u' What?',
 u' (After a moment, perks up).',
 u' Ow!',
 u" I told you, I'm no - (Gets hit again).",
 u" You're about to find out.",
 u' (Stands up).']

In [63]:
episode_three = pd.DataFrame.from_dict(speakers3, orient="index")
print episode_three.shape

(34, 24)


In [64]:
#from numpy import nan
#episode_three.fillna(value=str(0), inplace=True)

In [65]:
#episode_three.info()

In [66]:
#episode_three.head()

In [67]:
test3 = episode_three[0].apply(remove_stops)
#test3

In [68]:
for column in episode_three.columns:
    for x in range(len(episode_three[column])):
        episode_three[column][x] = remove_stops(episode_three[column][x])

In [69]:
#episode_three.info()

In [70]:
episode_three.index = episode_three.index.str.title()

episode_three.shape

(34, 24)

In [71]:
episode_three.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Stannis,olly closes door,well,say word,im giving chance avenge family,bored us already,know,theyd rather burn fight,youre lord commander,many enemies castle black,whoever said didnt many enemies,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Davos,sees something,king complicated man,hes one true king,,long boltons rule north stands north suffer,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Meryn,dont think good idea grace,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Pycelle,mans private affairs ought stay private,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Priestess,valyrian night dark full terrors,valyrian,valyrian bought sold varys approaches tyrion s...,valyrian sent saviour,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Four

In [72]:
#df_s5.script_text[3]

In [73]:
script_text5_04 = df_s5.script_text[3]
print type(script_text5_04)

#script_text5_04

<type 'list'>


In [74]:
script_split4 = [i.split('\n') for i in script_text5_04]
#script_split4

In [75]:
string_script_text4 = ' '

for i in script_split4:
    for e in i:
        string_script_text4 = string_script_text4 + ' ' + e

In [76]:
#string_script_text4

In [77]:
words4 = word_tokenize(string_script_text4)
#words4

In [78]:
sentences4 = sent_tokenize(string_script_text4)
#sentences4

In [79]:
dialogue4 = []

for sentence in sentences4:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names5_04):
            dialogue4.append(sentence)

In [80]:
#dialogue4

In [81]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s504.pkl', 'wb') as f:
    pickle.dump(dialogue4, f)

In [82]:
from collections import defaultdict

speakers4 = defaultdict(list)
for sentence in dialogue4:
    speakers4[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers4

In [83]:
speakers4['JON']

[u' (To some of the men sparring).',
 u' Never even heard of these people.',
 u' And how many men does this Lord Maison have to send us?',
 u' Not him.',
 u' He murdered my brother.',
 u' How can I help you?',
 u' Castle Black is my home now.',
 u" You're gonna show me some vision in the fire.",
 u" I don't think Stannis would like that very much.",
 u" I can't."]

In [84]:
episode_four = pd.DataFrame.from_dict(speakers4, orient="index")
print episode_four.shape

(36, 29)


In [85]:
#from numpy import nan
#episode_four.fillna(value=str(0), inplace=True)

In [86]:
#episode_four.info()

In [87]:
#episode_four.head()

In [88]:
test4 = episode_four[0].apply(remove_stops)
#test4

In [89]:
for column in episode_four.columns:
    for x in range(len(episode_four[column])):
        episode_four[column][x] = remove_stops(episode_four[column][x])

In [90]:
episode_four.index = episode_four.index.str.title()

episode_four.shape

(36, 29)

In [91]:
episode_four.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
Stannis,lord commander nights watch,fault,must,wont,walking away,father used tell boredom indicates lack inner ...,know castle black place child shireen,say,infant stands walks around desk dornish traile...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Elaria,nym obara,prince doran weep father nothing else,hes come myrcella,nym,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Selyse,given son,whose,lady,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Pycelle,small council grows smaller smaller,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Dario,entering,fifty hundred,think protect hizdahr zo loraq,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Five

In [92]:
#df_s5.script_text[4]

In [93]:
script_text5_05 = df_s5.script_text[4]
print type(script_text5_05)

#script_text5_05

<type 'list'>


In [94]:
script_split5 = [i.split('\n') for i in script_text5_05]
#script_split5

In [95]:
string_script_text5 = ' '

for i in script_split5:
    for e in i:
        string_script_text5 = string_script_text5 + ' ' + e

In [96]:
#string_script_text5

In [97]:
words5 = word_tokenize(string_script_text5)
#words5

In [98]:
sentences5 = sent_tokenize(string_script_text5)
#sentences5

In [99]:
dialogue5 = []

for sentence in sentences5:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names5_05):
            dialogue5.append(sentence)

In [100]:
#dialogue5

In [101]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s505.pkl', 'wb') as f:
    pickle.dump(dialogue5, f)

In [102]:
from collections import defaultdict

speakers5 = defaultdict(list)
for sentence in dialogue5:
    speakers5[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers5

In [103]:
speakers5['AEMON']

[u'   Her last relation thousands of miles away, useless, dying.',
 u'     JON',
 u' Lord Commander.',
 u' Oh, like a hundred-year-old man slowly freezing to death.',
 u' Half the men hate you already, Lord Commander.',
 u" That doesn't matter."]

In [104]:
episode_five = pd.DataFrame.from_dict(speakers5, orient="index")
print episode_five.shape

(26, 30)


In [105]:
#from numpy import nan
#episode_five.fillna(value=str(0), inplace=True)

In [106]:
#episode_five.info()

In [107]:
#episode_five.head()

In [108]:
test5 = episode_five[0].apply(remove_stops)
#test5

In [109]:
for column in episode_five.columns:
    for x in range(len(episode_five[column])):
        episode_five[column][x] = remove_stops(episode_five[column][x])

In [110]:
episode_five.index = episode_five.index.str.title()

episode_five.shape

(26, 30)

In [111]:
episode_five.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Stannis,nothing,youre samwell tarly,dont look like soldier,,dragonglass,lady melisandre told death marches wall,seen,jon snow returns wildlings,need theyre coming us,hope know youre wildlings,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Davos,,uh grace,ill choose dozen men stay guard queen princess,tough road ahead grace,beg pardon,aye princess wont anywhere near selyse,yes queen,well,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Gilly,world,grew books,wheres,im sorry dont know things,wash linens,like maester aemon,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Tyrion,would make time together truly enjoyable would...,person drinks,sailed route,youre going bring daenerys souvenir ancestral ...,demons flames,oh,smoking sea,werent,suppose,,mormont cut free,mormont,shakes head,itd kinder put daggers hearts done,without luck,thats best idea youve day,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Missandei,valyrian three days,valyrian failed one,men fear death,grace think fit opinion matters,tell seen grace,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Six

In [112]:
#df_s5.script_text[5]

In [113]:
script_text5_06 = df_s5.script_text[5]
print type(script_text5_06)

#script_text5_06

<type 'list'>


In [114]:
script_split6 = [i.split('\n') for i in script_text5_06]
#script_split6

In [115]:
string_script_text6 = ' '

for i in script_split6:
    for e in i:
        string_script_text6 = string_script_text6 + ' ' + e

In [116]:
#string_script_text6

In [117]:
words6 = word_tokenize(string_script_text6)
#words6

In [118]:
sentences6 = sent_tokenize(string_script_text6)
#sentences6

In [119]:
dialogue6 = []

for sentence in sentences6:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names5_06):
            dialogue6.append(sentence)

In [120]:
#dialogue6

In [121]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s506.pkl', 'wb') as f:
    pickle.dump(dialogue6, f)

In [122]:
from collections import defaultdict

speakers6 = defaultdict(list)
for sentence in dialogue6:
    speakers6[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers6

In [123]:
speakers6['ARYA']

[u' No!',
 u' When?',
 u' When do I get to play the game?',
 u' The game.',
 u' I didn\u2019t even know I was playing.',
 u' No one.',
 u' Well who are you?',
 u' What?',
 u' Arya.',
 u' Westeros.',
 u' After he was executed I fled the capital.',
 u' I stabbed him in the gut.',
 u' A man called the Hound.',
 u' Yes.',
 u' Valar dohaeris.',
 u'          INT.']

In [124]:
episode_six = pd.DataFrame.from_dict(speakers6, orient="index")
print episode_six.shape

(29, 24)


In [125]:
#from numpy import nan
#episode_six.fillna(value=str(0), inplace=True)

In [126]:
#episode_six.info()

In [127]:
#episode_six.head()

In [128]:
test6 = episode_six[0].apply(remove_stops)
#test6

In [129]:
for column in episode_six.columns:
    for x in range(len(episode_six[column])):
        episode_six[column][x] = remove_stops(episode_six[column][x])

In [130]:
episode_six.index = episode_six.index.str.title()

episode_six.shape

(29, 24)

In [131]:
episode_six.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Bronn,im coming best part,jaime,got princess,explains golden hand,dont let time alone son,lets something stupid,something stupid,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Jaime,trying blend,thats water gardens,like improvise,myrcella,lets speak private,excellent,go myrcella,youll fine promise bronn,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Tyrion,youre hungry,youre awful travelling companion know,travelling together,jorah,never occurred ask,killed father,miserable mormont least father good man,asked men knew stories every one,know heard jorah stronger,leading expedition beyond wall,believe many things,,,doesnt mean shes going great queen,conquers world,,thats justice,father burned living men amusement king,wait wait,wait wait wait wait,malko,fighting puts mereen,looks jorah,tyrion
Malko,galley slave,worthless,chop cock,bound volantis slavers bay,,jousting,liar,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Girl,hurts,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Seven

In [132]:
#df_s5.script_text[6]

In [133]:
script_text5_07 = df_s5.script_text[6]
print type(script_text5_07)

#script_text5_07

<type 'list'>


In [134]:
script_split7 = [i.split('\n') for i in script_text5_07]
#script_split7

In [135]:
string_script_text7 = ' '

for i in script_split7:
    for e in i:
        string_script_text7 = string_script_text7 + ' ' + e

In [136]:
#string_script_text7

In [137]:
words7 = word_tokenize(string_script_text7)
#words7

In [138]:
sentences7 = sent_tokenize(string_script_text7)
#sentences7

In [139]:
dialogue7 = []

for sentence in sentences7:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names5_07):
            dialogue7.append(sentence)

In [140]:
#dialogue7

In [141]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s507.pkl', 'wb') as f:
    pickle.dump(dialogue7, f)

In [142]:
from collections import defaultdict

speakers7 = defaultdict(list)
for sentence in dialogue7:
    speakers7[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers7

In [143]:
speakers7['BAELISH']

[u' I\u2019m sorry about the locale.',
 u' It felt like the safest place.',
 u' It was an establishment like no other.',
 u' The past is the past.',
 u' I promise you..    OLENNA',
 u' Cersei summoned me to King\u2019s Landing.',
 u' A piece of info that she know that I had.']

In [144]:
episode_seven = pd.DataFrame.from_dict(speakers7, orient="index")
print episode_seven.shape

(36, 20)


In [145]:
#from numpy import nan
#episode_seven.fillna(value=str(0), inplace=True)

In [146]:
#episode_seven.info()

In [147]:
#episode_seven.head()

In [148]:
test7 = episode_seven[0].apply(remove_stops)
#test7

In [149]:
for column in episode_seven.columns:
    for x in range(len(episode_seven[column])):
        episode_seven[column][x] = remove_stops(episode_seven[column][x])

In [150]:
episode_seven.index = episode_seven.index.str.title()

episode_seven.shape

(36, 20)

In [152]:
episode_seven.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Stannis,else,sell swords loyal nothing gold,,davos,right time risk everything,dont know saw,trust,sure,dont roberts bastard,must another way,shes daughter,get,None,None,None,None,None,None,None,None
Davos,stannis,still hard march wintefell wont marching anywh...,isnt time head back castle black snow clears,better wait right time risk everything,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Jorah,khaleesi please need moment time,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Slave Buyer,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Melisandre,saw king stared flames,know,trust lord,seen walk along battlements wintefell,sometimes sacrifices must made ensure victory,stannis,doubt still,one way,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Eight

In [153]:
#df_s5.script_text[7]

In [154]:
script_text5_08 = df_s5.script_text[7]
print type(script_text5_08)

#script_text5_08

<type 'list'>


In [155]:
script_split8 = [i.split('\n') for i in script_text5_08]
#script_split8

In [156]:
string_script_text8 = ' '

for i in script_split8:
    for e in i:
        string_script_text8 = string_script_text8 + ' ' + e

In [157]:
#string_script_text8

In [158]:
words8 = word_tokenize(string_script_text8)
#words8

In [159]:
sentences8 = sent_tokenize(string_script_text8)
#sentences8

In [160]:
dialogue8 = []

for sentence in sentences8:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names5_08):
            dialogue8.append(sentence)

/Users/zoemarkovits/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [161]:
#dialogue8

In [162]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s508.pkl', 'wb') as f:
    pickle.dump(dialogue8, f)

In [163]:
from collections import defaultdict

speakers8 = defaultdict(list)
for sentence in dialogue8:
    speakers8[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers8

In [164]:
speakers8['ARYA']

[u' How will she serve him?',
 u' What will she do there?',
 u' See what?',
 u'         ARYA',
 u' Best in the city.',
 u' So why would a captain make the wager in the first place?',
 u' If the captain dies, the thin man pays his family a lot of money.',
 u' The Many Faced God.',
 u' And then what?']

In [165]:
episode_eight = pd.DataFrame.from_dict(speakers8, orient="index")
print episode_eight.shape

(27, 31)


In [165]:
#from numpy import nan
#episode_eight.fillna(value=str(0), inplace=True)

In [166]:
#episode_eight.info()

In [167]:
#episode_eight.head()

In [168]:
test8 = episode_eight[0].apply(remove_stops)
#test8

In [169]:
for column in episode_eight.columns:
    for x in range(len(episode_eight[column])):
        episode_eight[column][x] = remove_stops(episode_eight[column][x])

In [170]:
episode_eight.index = episode_eight.index.str.title()

episode_eight.shape

(27, 31)

In [171]:
episode_eight.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
Karsi,built wall keep us,beat white walkers,old stories dragonglass,things weve seen dont believe,come,,lost father uncle brothers fighting damn crows,vouch man tormund,would mine fuck em theyre dead,johnna gonna look,need get old folks boats,,dont think youre gonna enforce orders,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Brother,youll never make jon,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Olly,olly,heard happened,figured hungry,wanted ask something,lord commander,hes going hardhome save wildlings,wasnt fight,one hes travelling one red beard led raid village,let wildings gates cut throats sleep,believe,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Tyrion,tyrion,daenerys,wealth lands army name handful supporters prob...,build better world,know,whomever ser jorah started informing longer man,opportunity confess betrayals,,worships,ruler kills devoted ruler inspires devotion,decided yet whether youre going killed,see would think,father,day decide execute ill tell killed father,daenerys,danerys,see right kind terrible,kind prevents people even,yes wise,yes,survive,thats one,queenly prerogative,,im going murdered im going banished daenerys,advise,iron throne,im entirely joking,get back home supports,lets generous assume thats going happen,daenerys
Daughter,wanna go,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Nine

In [172]:
#df_s5.script_text[8]

In [173]:
script_text5_09 = df_s5.script_text[8]
print type(script_text5_09)

#script_text5_09

<type 'list'>


In [174]:
script_split9 = [i.split('\n') for i in script_text5_09]
#script_split9

In [175]:
string_script_text9 = ' '

for i in script_split9:
    for e in i:
        string_script_text9 = string_script_text9 + ' ' + e

In [176]:
#string_script_text9

In [177]:
words9 = word_tokenize(string_script_text9)
#words9

In [178]:
sentences9 = sent_tokenize(string_script_text9)
#sentences9

In [181]:
dialogue9 = []

for sentence in sentences9:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names5_09):
            dialogue9.append(sentence)

In [182]:
#dialogue9

In [183]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s509.pkl', 'wb') as f:
    pickle.dump(dialogue9, f)

In [184]:
from collections import defaultdict

speakers9 = defaultdict(list)
for sentence in dialogue9:
    speakers9[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers9

In [185]:
speakers9['ARYA']

[u' Oysters, clams, and cockels!', u'  .', u' Tomorrow.']

In [186]:
episode_nine = pd.DataFrame.from_dict(speakers9, orient="index")
print episode_nine.shape

(36, 16)


In [187]:
#from numpy import nan
#episode_nine.fillna(value=str(0), inplace=True)

In [188]:
#episode_nine.info()

In [189]:
#episode_nine.head()

In [190]:
test9 = episode_nine[0].apply(remove_stops)
#test9

In [192]:
for column in episode_nine.columns:
    for x in range(len(episode_nine[column])):
        episode_nine[column][x] = remove_stops(episode_nine[column][x])

In [193]:
episode_nine.index = episode_nine.index.str.title()

episode_nine.shape

(36, 16)

In [194]:
episode_nine.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Stannis,put last nights guards chains either fell asle...,returning castle black,find healthy horses good men guard im sending ...,yes,youre abandoning youre obeying command,jon snow refuses boy scroll boy say,family stays,whats,dance dragons,doesnt make much sense,choose rhaenyra aegon would chosen,dont even know im talking,yes,ext,melisandre,None
Davos,stannis,northerners know land ever,unless theres thaw cant press forward winterfe...,forgive grace never claimed expert military ma...,grace youve named hand,kings hand never abandon king especially time war,boy scroll could deliver message,grace perhaps queen princess shireen could acc...,stannis,thus ending dragonslaying career ser byron swann,like,youre welcome,course,deserve,None,None
Selyse,lord wants,dont act well starve us make sacrifice melisandre,daenerys,None,None,None,None,None,None,None,None,None,None,None,None,None
Meryn,meryn,oh im buying,old,old,want,good,ill tell done,youll fresh one tomorrow,None,None,None,None,None,None,None,None
Announcer,queen first contest,meereenese champion,westerosi knight,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Ten

In [195]:
#df_s5.script_text[9]

In [196]:
script_text5_10 = df_s5.script_text[9]
print type(script_text5_10)

#script_text5_10

<type 'list'>


In [197]:
script_split10 = [i.split('\n') for i in script_text5_10]
#script_split10

In [198]:
string_script_text10 = ' '

for i in script_split10:
    for e in i:
        string_script_text10 = string_script_text10 + ' ' + e

In [199]:
#string_script_text10

In [200]:
words10 = word_tokenize(string_script_text10)
#words10

In [201]:
sentences10 = sent_tokenize(string_script_text10)
#sentences10

In [202]:
dialogue10 = []

for sentence in sentences10:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names5_10):
            dialogue10.append(sentence)

In [203]:
#dialogue10

In [201]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s510.pkl', 'wb') as f:
    pickle.dump(dialogue10, f)

In [204]:
from collections import defaultdict

speakers10 = defaultdict(list)
for sentence in dialogue10:
    speakers10[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers10

In [205]:
speakers10['ARYA']

[u' You were the first person on my list, you know.',
 u' I can\u2019t hear you.',
 u' You know who I am.',
 u' Do you know who you are?',
 u' You\u2019re no one.',
 u' No!',
 u' He was my friend.',
 u' But if you\u2019re\u2026 (looks to the Jaqen who drank the potion and is seemingly dead) Who\u2019s this?',
 u' What\u2019s happening?!']

In [206]:
episode_ten = pd.DataFrame.from_dict(speakers10, orient="index")
print episode_ten.shape

(38, 16)


In [207]:
#from numpy import nan
#episode_ten.fillna(value=str(0), inplace=True)

In [208]:
#episode_ten.info()

In [209]:
#episode_ten.head()

In [210]:
test10 = episode_ten[0].apply(remove_stops)
#test10

In [211]:
for column in episode_ten.columns:
    for x in range(len(episode_ten[column])):
        episode_ten[column][x] = remove_stops(episode_ten[column][x])

In [212]:
episode_ten.index = episode_ten.index.str.title()

episode_ten.shape

(38, 16)

In [213]:
episode_ten.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Stannis,ride winterfell,prepare form,tell,many,speak,int,bolton women fighting,None,None,None,None,None,None,None,None,None
Davos,wildlings make difference,shireen,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Melisandre,take,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Brother,watch,watch,watch,watch,None,None,None,None,None,None,None,None,None,None,None,None
Bronn,maybe ill come visit sometime,dont wait long,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Season Five

In [214]:
episode_one.index

Index([u'Stannis', u'Davos', u'Kevan', u'Melisandre', u'Olly', u'Jaime',
       u'Tyrion', u'Missandei', u'Mossador', u'Prostitute', u'Baelish',
       u'Mance', u'Sam', u'Grand Maester Pyrcelle', u'Daario',
       u'Kevan Lannister', u'Daenerys', u'Jon', u'Barristan', u'Hizdahr',
       u'Melara', u'Grey Worm', u'Cersei', u'Varys', u'Alliser', u'Maggy',
       u'White Rat', u'Margaery', u'Gilly', u'Podrick', u'Lancel', u'Sansa',
       u'Brienne', u'Squire', u'Olyvar', u'Yohn Royce', u'Priest', u'Loras',
       u'Royce'],
      dtype='object')

In [215]:
episode_two.index

Index([u'Stannis', u'Davos', u'Shireen', u'Meryn', u'Pycelle', u'Kevan',
       u'Old Man', u'Brother', u'Bronn', u'Gilly', u'Jaime', u'Tyrion',
       u'Lollys', u'Mossador', u'Baelish', u'Ellaria', u'Sam', u'Areo',
       u'Mace Tyrell', u'Guard', u'Selyse', u'Daenerys', u'Doran', u'Aemon',
       u'Jon', u'Barristan', u'Brothers', u'Hunters', u'Grey Worm', u'Cersei',
       u'Varys', u'Mace', u'Janos', u'Qyburn', u'Hizdahr', u'Podrick',
       u'Sansa', u'Brienne', u'Knight', u'Men', u'Jaqen', u'Daario',
       u'Alliser', u'Waitress', u'Arya', u'Knight 1', u'Man'],
      dtype='object')

In [216]:
print episode_one.shape
print episode_two.shape

(39, 22)
(47, 23)


In [217]:
episode_one = episode_one.reset_index()

In [218]:
episode_one_dict = {'Stannis':'Stannis', 
                    'Davos':'Davos', 
                    'Kevan':'Kevan', 
                    'Melisandre':'Melisandre', 
                    'Olly':'Olly', 
                    'Jaime':'Jaime',
                    'Tyrion':'Tyrion', 
                    'Missandei':'Missandei', 
                    'Mossador':'Mossador', 
                    'Prostitute':'Prostitute', 
                    'Baelish':'Baelish',
                    'Mance':'Mance', 
                    'Sam':'Sam', 
                    'Grand Maester Pyrcelle':'Pycelle', 
                    'Daario':'Daario',
                    'Kevan Lannister':'Kevan', 
                    'Daenerys':'Daenerys', 
                    'Jon':'Jon', 
                    'Barristan':'Barristan', 
                    'Hizdahr':'Hizdahr',
                    'Melara':'Melara', 
                    'Grey Worm':'Grey Worm', 
                    'Cersei':'Cersei', 
                    'Varys':'Varys', 
                    'Alliser':'Alliser', 
                    'Maggy':'Maggy',
                    'White Rat':'White Rat', 
                    'Margaery':'Margaery', 
                    'Gilly':'Gilly', 
                    'Podrick':'Podrick', 
                    'Lancel':'Lancel', 
                    'Sansa':'Sansa',
                    'Brienne':'Brienne', 
                    'Squire':'Squire', 
                    'Olyvar':'Olyvar', 
                    'Yohn Royce':'Yohn Royce', 
                    'Priest':'Priest', 
                    'Loras':'Loras',
                    'Royce':'Royce'}

episode_one['index']=episode_one['index'].replace(episode_one_dict)

In [219]:
episode_one = episode_one.set_index('index')

In [222]:
#episode_one.head()

In [223]:
season_five = episode_one.join(episode_two, rsuffix="_two", how='outer')
print season_five.shape

(64, 45)


In [224]:
#season_five.head(65)

In [225]:
season_five.index

Index([      u'Aemon',     u'Alliser',        u'Areo',        u'Arya',
           u'Baelish',   u'Barristan',     u'Brienne',       u'Bronn',
           u'Brother',    u'Brothers',      u'Cersei',      u'Daario',
          u'Daenerys',       u'Davos',       u'Doran',     u'Ellaria',
             u'Gilly',   u'Grey Worm',       u'Guard',     u'Hizdahr',
           u'Hunters',       u'Jaime',       u'Janos',       u'Jaqen',
               u'Jon',       u'Kevan',       u'Kevan',      u'Knight',
          u'Knight 1',      u'Lancel',      u'Lollys',       u'Loras',
              u'Mace', u'Mace Tyrell',       u'Maggy',         u'Man',
             u'Mance',    u'Margaery',      u'Melara',  u'Melisandre',
               u'Men',       u'Meryn',   u'Missandei',    u'Mossador',
           u'Old Man',        u'Olly',      u'Olyvar',     u'Podrick',
            u'Priest',  u'Prostitute',     u'Pycelle',      u'Qyburn',
             u'Royce',         u'Sam',       u'Sansa',      u'Selyse',
      

In [226]:
episode_three.index

Index([u'Stannis', u'Davos', u'Meryn', u'Pycelle', u'Priestess', u'Olly',
       u'Tyrion', u'Baelish', u'Crowd', u'Janos', u'Roose', u'Jon', u'Woman 2',
       u'Man 1', u'Tommen', u'Man 2', u'Cersei', u'Varys', u'Waif', u'Mace',
       u'Woman', u'Margaery', u'Qyburn', u'Ramsay', u'Man 3', u'Podrick',
       u'Sam', u'Sansa', u'Jaqen', u'Olyvar', u'Brienne', u'Arya',
       u'High Septon', u'Man'],
      dtype='object')

In [227]:
episode_three.shape

(34, 24)

In [228]:
season_five = season_five.join(episode_three, rsuffix="_three", how='outer')
print season_five.shape

(76, 69)


In [229]:
#season_five.index

In [230]:
episode_four.index

Index([u'Stannis', u'Elaria', u'Selyse', u'Pycelle', u'Dario', u'Melisandre',
       u'Bronn', u'Jaime', u'Tyrion', u'Missandei', u'Woman', u'Sam',
       u'Shireen', u'Kingsguard', u'Man 3', u'Jon', u'Barristan', u'Man 1',
       u'Tommen', u'Man 2', u'Cersei', u'Man 4', u'Mace', u'Baelish',
       u'Daenerys', u'Margaery', u'Rider', u'Nymeria', u'Hizdahr', u'Lancel',
       u'Obara', u'Sansa', u'Jorah', u'Militant', u'High Sparrow', u'Loras'],
      dtype='object')

In [231]:
episode_four = episode_four.reset_index()

In [232]:
episode_four_dict = {'Stannis':'Stannis', 
                     'Elaria':'Ellaria', 
                     'Selyse':'Selyse', 
                     'Pycelle':'Pycelle', 
                     'Dario':'Daario', 
                     'Melisandre':'Melisandre',
                     'Bronn':'Bronn', 
                     'Jaime':'Jaime', 
                     'Tyrion':'Tyrion', 
                     'Missandei':'Missandei', 
                     'Woman':'Woman', 
                     'Sam':'Sam',
                     'Shireen':'Shireen', 
                     'Kingsguard':'Kingsguard', 
                     'Man 3':'Man 3', 
                     'Jon':'Jon', 
                     'Barristan':'Barristan', 
                     'Man 1':'Man 1',
                     'Tommen':'Tommen', 
                     'Man 2':'Man 2', 
                     'Cersei':'Cersei', 
                     'Man 4':'Man 4', 
                     'Mace':'Mace', 
                     'Baelish':'Baelish',
                     'Daenerys':'Daenerys', 
                     'Margaery':'Margaery', 
                     'Rider':'Rider', 
                     'Nymeria':'Nymeria', 
                     'Hizdahr':'Hizdahr', 
                     'Lancel':'Lancel',
                     'Obara':'Obara', 
                     'Sansa':'Sansa', 
                     'Jorah':'Jorah', 
                     'Militant':'Militant', 
                     'High Sparrow':'High Sparrow', 
                     'Loras':'Loras'}

episode_four['index'] = episode_four['index'].replace(episode_four_dict)

In [233]:
episode_four = episode_four.set_index('index')

In [234]:
episode_four.shape

(36, 29)

In [235]:
season_five = season_five.join(episode_four, rsuffix="_four", how='outer')
print season_five.shape

(84, 98)


In [236]:
#season_five.index

In [237]:
episode_five.index

Index([u'Stannis', u'Davos', u'Gilly', u'Tyrion', u'Missandei', u'Sam',
       u'Olly', u'Shireen', u'Daenerys', u'Roose', u'Jon', u'Myranda',
       u'Walda', u'Grey Worm', u'Theon', u'Ramsay', u'Hizdahr', u'Edd',
       u'Sansa', u'Brienne', u'Jorah', u'Allister', u'Aemon', u'Daario',
       u'Tormund', u'Man'],
      dtype='object')

In [238]:
episode_five = episode_five.reset_index()

In [239]:
episode_five_dict = {'Stannis':'Stannis', 
                     'Davos':'Davos', 
                     'Gilly':'Gilly', 
                     'Tyrion':'Tyrion', 
                     'Missandei':'Missandei', 
                     'Sam':'Sam',
                     'Olly':'Olly', 
                     'Shireen':'Shireen', 
                     'Daenerys':'Daenerys', 
                     'Roose':'Roose', 
                     'Jon':'Jon', 
                     'Myranda':'Myranda',
                     'Walda':'Walda', 
                     'Grey Worm':'Grey Worm', 
                     'Theon':'Theon', 
                     'Ramsay':'Ramsay', 
                     'Hizdahr':'Hizdahr', 
                     'Edd':'Edd',
                     'Sansa':'Sansa', 
                     'Brienne':'Brienne', 
                     'Jorah':'Jorah', 
                     'Allister':'Alliser', 
                     'Aemon':'Aemon', 
                     'Daario':'Daario',
                     'Tormund':'Tormund', 
                     'Man':'Man'}

episode_five['index'] = episode_five['index'].replace(episode_five_dict)

In [240]:
episode_five = episode_five.set_index('index')

In [241]:
episode_five.shape

(26, 30)

In [242]:
season_five = season_five.join(episode_five, rsuffix="_five", how='outer')
print season_five.shape    

(89, 128)


In [243]:
#season_five.index

In [244]:
episode_six.index

Index([u'Bronn', u'Jaime', u'Tyrion', u'Malko', u'Girl', u'Baelish',
       u'Sparrow', u'Areo', u'Theon', u'Roose', u'Myranda', u'Slaver', u'Waif',
       u'Sand Snakes', u'Cersei', u'Trystane', u'Margaery', u'Ramsay',
       u'Lancel', u'Obara', u'Sansa', u'Jaqen', u'Jorah', u'Olyvar', u'Olenna',
       u'Arya', u'Myrcella', u'Loras', u'Man'],
      dtype='object')

In [245]:
episode_six.shape

(29, 24)

In [246]:
season_five = season_five.join(episode_six, rsuffix="_six", how='outer')
print season_five.shape 

(97, 152)


In [247]:
#season_five.index

In [248]:
episode_seven.index

Index([u'Stannis', u'Davos', u'Jorah', u'Slave Buyer', u'Melisandre', u'Bronn',
       u'Gilly', u'Jaime', u'Tyrion', u'Malko', u'Baelish', u'Sam', u'Sparrow',
       u'Olenna', u'Hizdahr', u'Daenerys', u'Aemon', u'Jon', u'Brothers',
       u'Theon', u'Tommen', u'Derryk', u'Cersei', u'Buyer', u'Alliser',
       u'Margaery', u'Ramsay', u'Tyene', u'Obara', u'Sansa', u'Brand', u'Men',
       u'Owner', u'Daario', u'High Sparrow', u'Myrcella'],
      dtype='object')

In [249]:
episode_seven.shape

(36, 20)

In [250]:
season_five = season_five.join(episode_seven, rsuffix="_seven", how='outer')
print season_five.shape 

(103, 172)


In [251]:
#season_five.index

In [252]:
episode_eight.index

Index([u'Karsi', u'Brother', u'Olly', u'Tyrion', u'Daughter', u'Sam',
       u'Master', u'Daenerys', u'Roose', u'Jon', u'Giant', u'Loboda', u'Theon',
       u'Cersei', u'Wildling Elder', u'Qyburn', u'Ramsay', u'Gilly', u'Edd',
       u'Sansa', u'Jaqen', u'Jorah', u'Septa', u'Thin Man', u'Tormund',
       u'Arya', u'Man'],
      dtype='object')

In [253]:
episode_eight.shape

(27, 31)

In [254]:
season_five = season_five.join(episode_eight, rsuffix="_eight", how='outer')
print season_five.shape 

(111, 203)


In [255]:
episode_nine.shape

(36, 16)

In [256]:
episode_nine.index

Index([u'Stannis', u'Davos', u'Selyse', u'Meryn', u'Announcer', u'Melisandre',
       u'Bronn', u'Tyene', u'Jaime', u'Tyrion', u'Quick', u'Strong',
       u'Prostitute', u'Ellaria', u'Sam', u'Areo', u'Shireen', u'Daenerys',
       u'Doran', u'Jon', u'Head', u'Mace', u'Alliser', u'Trystane', u'Nymeria',
       u'Hizdahr', u'John', u'Jaqen', u'Jorah', u'Thin Man', u'Daario',
       u'Banker', u'Arya', u'Head Prostitute', u'Myrcella', u'Man'],
      dtype='object')

In [257]:
season_five = season_five.join(episode_nine, rsuffix="_nine", how='outer')
print season_five.shape 

(118, 219)


In [258]:
episode_ten.shape

(38, 16)

In [259]:
episode_ten.index

Index([u'Stannis', u'Davos', u'Melisandre', u'Brother', u'Bronn', u'Tyene',
       u'Soldier', u'Jaime', u'Tyrion', u'Missandei', u'Woman', u'Ellaria',
       u'Survivor', u'Sparrow', u'Theon', u'Daenerys', u'Jon', u'Myranda',
       u'Grey Worm', u'Waif', u'Varys', u'Cersei', u'Alliser', u'Qyburn',
       u'Ramsay', u'Olly', u'Podrick', u'Sam', u'Sansa', u'Brienne', u'Jorah',
       u'Septa', u'Jaqen', u'Daario', u'High Sparrow', u'Arya', u'Myrcella',
       u'Man'],
      dtype='object')

In [260]:
season_five = season_five.join(episode_ten, rsuffix="_ten", how='outer')
print season_five.shape 

(120, 235)


In [261]:
#season_five.index

In [262]:
from numpy import nan
season_five.fillna(value=str(0), inplace=True)

In [263]:
#season_five.head(125)

In [264]:
season_five.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, Aemon to Yohn Royce
Columns: 235 entries, 0 to 15_ten
dtypes: object(235)
memory usage: 221.2+ KB


In [265]:
season_five.to_pickle('/Users/zoemarkovits/Metis/project_five_got/PKLs/season_five.pkl')

## Character Breakdown

### Arya Stark

In [266]:
Arya = season_five.loc['Arya']
#print Arya

In [267]:
Arya_Text = Arya.values.tolist()
#Arya_Text

In [268]:
Arya_Words = []

for word in Arya_Text:
    if len(word)>=1:
        if word != '0':
            Arya_Words.append(word)
        
#Arya_Words 

In [269]:
Arya_Words_String = ' '.join(Arya_Words)
#Arya_Words_String

In [270]:
Arya_String_Length = len(Arya_Words_String)
Arya_String_Length

673

In [271]:
Arya_Sentiment = TextBlob(Arya_Words_String).sentiment
Arya_Sentiment

Sentiment(polarity=0.07499999999999998, subjectivity=0.4395833333333333)

### Petyr Baelish/Littlefinger

In [272]:
Littlefinger = season_five.loc['Baelish']
#print Littlefinger

In [273]:
Littlefinger_Text = Littlefinger.values.tolist()
#Littlefinger_Text

In [274]:
Littlefinger_Words = []

for word in Littlefinger_Text:
    if len(word)>=1:
        if word != '0':
            Littlefinger_Words.append(word)
        
#Littlefinger_Words 

In [275]:
Littlefinger_Words_String = ' '.join(Littlefinger_Words)
#Littlefinger_Words_String

In [276]:
Littlefinger_String_Length = len(Littlefinger_Words_String)
Littlefinger_String_Length

1456

In [277]:
Littlefinger_Sentiment = TextBlob(Littlefinger_Words_String).sentiment
Littlefinger_Sentiment

Sentiment(polarity=-0.02623753058535667, subjectivity=0.5615847920195747)

### Brienne of Tarth

In [278]:
Brienne = season_five.loc['Brienne']
#print Brienne

In [279]:
Brienne_Text = Brienne.values.tolist()
#Brienne_Text

In [280]:
Brienne_Words = []

for word in Brienne_Text:
    if len(word)>=1:
        if word != '0':
            Brienne_Words.append(word)
        
#Brienne_Words 

In [281]:
Brienne_Words_String = ' '.join(Brienne_Words)
#Brienne_Words_String

In [282]:
Brienne_String_Length = len(Brienne_Words_String)
Brienne_String_Length

881

In [283]:
Brienne_Sentiment = TextBlob(Brienne_Words_String).sentiment
Brienne_Sentiment

Sentiment(polarity=0.07142857142857142, subjectivity=0.5507936507936508)

### Cersei Lannister

In [284]:
Cersei = season_five.loc['Cersei']
#print Cersei

In [285]:
Cersei_Text = Cersei.values.tolist()
#Cersei_Text

In [286]:
Cersei_Words = []

for word in Cersei_Text:
    if len(word)>=1:
        if word != '0':
            Cersei_Words.append(word)
        
#Cersei_Words 

In [287]:
Cersei_Words_String = ' '.join(Cersei_Words)
#Cersei_Words_String

In [288]:
Cersei_String_Length = len(Cersei_Words_String)
Cersei_String_Length

3292

In [289]:
Cersei_Sentiment = TextBlob(Cersei_Words_String).sentiment
Cersei_Sentiment

Sentiment(polarity=0.033030303030303035, subjectivity=0.6062218678008151)

### Daenerys Targaryan

In [290]:
Daenerys = season_five.loc['Daenerys']
#print Daenerys

In [291]:
Dany_Text = Daenerys.values.tolist()
#Dany_Text

In [292]:
Dany_Words = []

for word in Dany_Text:
    if len(word)>=1:
        if word != '0':
            Dany_Words.append(word)
        
#Dany_Words 

In [293]:
Dany_Words_String = ' '.join(Dany_Words)
#Dany_Words_String

In [294]:
Dany_String_Length = len(Dany_Words_String)
Dany_String_Length

1839

In [295]:
Dany_Sentiment = TextBlob(Dany_Words_String).sentiment
Dany_Sentiment

Sentiment(polarity=0.1770833333333333, subjectivity=0.5620370370370371)

### Davos Seaworth

In [296]:
Davos = season_five.loc['Davos']
#print Davos

In [297]:
Davos_Text = Davos.values.tolist()
#Davos_Text

In [298]:
Davos_Words = []

for word in Davos_Text:
    if len(word)>=1:
        if word != '0':
            Davos_Words.append(word)
        
#Davos_Words 

In [299]:
Davos_Words_String = ' '.join(Davos_Words)
#Davos_Words_String

In [300]:
Davos_String_Length = len(Davos_Words_String)
Davos_String_Length

1005

In [301]:
Davos_Sentiment = TextBlob(Davos_Words_String).sentiment
Davos_Sentiment

Sentiment(polarity=0.06774734926908842, subjectivity=0.5709533220402786)

### Gilly

In [302]:
Gilly = season_five.loc['Gilly']
#print Gilly

In [303]:
Gilly_Text = Gilly.values.tolist()
#Gilly_Text

In [304]:
Gilly_Words = []

for word in Gilly_Text:
    if len(word)>=1:
        if word != '0':
            Gilly_Words.append(word)
        
#Gilly_Words 

In [305]:
Gilly_Words_String = ' '.join(Gilly_Words)
#Gilly_Words_String

In [306]:
Gilly_String_Length = len(Gilly_Words_String)
Gilly_String_Length

466

In [307]:
Gilly_Sentiment = TextBlob(Gilly_Words_String).sentiment
Gilly_Sentiment

Sentiment(polarity=0.10357142857142856, subjectivity=0.7428571428571428)

### Jaime Lannister

In [308]:
Jaime = season_five.loc['Jaime']
#print Jaime

In [309]:
Jaime_Text = Jaime.values.tolist()
#Jaime_Text

In [310]:
Jaime_Words = []

for word in Jaime_Text:
    if len(word)>=1:
        if word != '0':
            Jaime_Words.append(word)
        
#Jaime_Words 

In [311]:
Jaime_Words_String = ' '.join(Jaime_Words)
#Jaime_Words_String

In [312]:
Jaime_String_Length = len(Jaime_Words_String)
#Jaime_String_Length

In [313]:
Jaime_Sentiment = TextBlob(Jaime_Words_String).sentiment
Jaime_Sentiment

Sentiment(polarity=0.22473118279569887, subjectivity=0.7362903225806453)

### Jon Snow

In [314]:
Jon = season_five.loc['Jon']
#print Jon

In [315]:
Jon_Text = Jon.values.tolist()
#Jon_Text

In [316]:
Jon_Words = []

for word in Jon_Text:
    if len(word)>=1:
        if word != '0':
            Jon_Words.append(word)
        
#Jon_Words 

In [317]:
Jon_Words_String = ' '.join(Jon_Words)
#Jon_Words_String

In [318]:
Jon_String_Length = len(Jon_Words_String)
Jon_String_Length

2145

In [319]:
Jon_Sentiment = TextBlob(Jon_Words_String).sentiment
Jon_Sentiment

Sentiment(polarity=0.10729564553093966, subjectivity=0.5183473389355743)

### Jorah Mormont

In [320]:
Jorah = season_five.loc['Jorah']
#print Jorah

In [321]:
Jorah_Text = Jorah.values.tolist()
#Jorah_Text

In [322]:
Jorah_Words = []

for word in Jorah_Text:
    if len(word)>=1:
        if word != '0':
            Jorah_Words.append(word)
        
#Jorah_Words 

In [323]:
Jorah_Words_String = ' '.join(Jorah_Words)
#Jorah_Words_String

In [324]:
Jorah_String_Length = len(Jorah_Words_String)
Jorah_String_Length

733

In [325]:
Jorah_Sentiment = TextBlob(Jorah_Words_String).sentiment
Jorah_Sentiment

Sentiment(polarity=0.11187641723356008, subjectivity=0.4519368858654572)

### Margaery Tyrell

In [326]:
Margaery = season_five.loc['Margaery']
#print Margaery

In [327]:
Margaery_Text = Margaery.values.tolist()
#Margaery_Text

In [328]:
Margaery_Words = []

for word in Margaery_Text:
    if len(word)>=1:
        if word != '0':
            Margaery_Words.append(word)
        
#Margaery_Words 

In [329]:
Margaery_Words_String = ' '.join(Margaery_Words)
#Margaery_Words_String

In [330]:
Margaery_String_Length = len(Margaery_Words_String)
Margaery_String_Length

835

In [331]:
Margaery_Sentiment = TextBlob(Margaery_Words_String).sentiment
Margaery_Sentiment

Sentiment(polarity=0.18622222222222226, subjectivity=0.7137777777777778)

### Melisandre

In [332]:
Melisandre = season_five.loc['Melisandre']
#print Melisandre

In [333]:
Melisandre_Text = Melisandre.values.tolist()
#Melisandre_Text

In [334]:
Melisandre_Words = []

for word in Melisandre_Text:
    if len(word)>=1:
        if word != '0':
            Melisandre_Words.append(word)
        
#Melisandre_Words 

In [335]:
Melisandre_Words_String = ' '.join(Melisandre_Words)
#Melisandre_Words_String

In [336]:
Melisandre_String_Length = len(Melisandre_Words_String)
Melisandre_String_Length

512

In [337]:
Melisandre_Sentiment = TextBlob(Melisandre_Words_String).sentiment
Melisandre_Sentiment

Sentiment(polarity=0.11499999999999999, subjectivity=0.38166666666666665)

### Missandei

In [338]:
Missandei = season_five.loc['Missandei']
#print Missandei

In [339]:
Missandei_Text = Missandei.values.tolist()
#Missandei_Text

In [340]:
Missandei_Words = []

for word in Missandei_Text:
    if len(word)>=1:
        if word != '0':
            Missandei_Words.append(word)
        
#Missandei_Words 

In [341]:
Missandei_Words_String = ' '.join(Missandei_Words)
#Missandei_Words_String

In [342]:
Missandei_String_Length = len(Missandei_Words_String)
Missandei_String_Length

203

In [343]:
Missandei_Sentiment = TextBlob(Missandei_Words_String).sentiment
Missandei_Sentiment

Sentiment(polarity=0.11388888888888889, subjectivity=0.4305555555555555)

### Olenna Tyrell

In [344]:
Olenna = season_five.loc['Olenna']
#print Olenna

In [345]:
Olenna_Text = Olenna.values.tolist()
#Olenna_Text

In [346]:
Olenna_Words = []

for word in Olenna_Text:
    if len(word)>=1:
        if word != '0':
            Olenna_Words.append(word)
        
#Olenna_Words 

In [347]:
Olenna_Words_String = ' '.join(Olenna_Words)
#Olenna_Words_String

In [348]:
Olenna_String_Length = len(Olenna_Words_String)
Olenna_String_Length

620

In [349]:
Olenna_Sentiment = TextBlob(Olenna_Words_String).sentiment
Olenna_Sentiment

Sentiment(polarity=-0.0033333333333333214, subjectivity=0.5816666666666667)

### Ramsay Bolton

In [350]:
Ramsay = season_five.loc['Ramsay']
#print Ramsay

In [351]:
Ramsay_Text = Ramsay.values.tolist()
#Ramsay_Text

In [352]:
Ramsay_Words = []

for word in Ramsay_Text:
    if len(word)>=1:
        if word != '0':
            Ramsay_Words.append(word)
        
#Ramsay_Words 

In [353]:
Ramsay_Words_String = ' '.join(Ramsay_Words)
#Ramsay_Words_String

In [354]:
Ramsay_String_Length = len(Ramsay_Words_String)
Ramsay_String_Length

933

In [355]:
Ramsay_Sentiment = TextBlob(Ramsay_Words_String).sentiment
Ramsay_Sentiment

Sentiment(polarity=0.08597763347763347, subjectivity=0.6397402597402596)

### Samwell Tarly

In [356]:
Sam = season_five.loc['Sam']
#print Sam

In [357]:
Sam_Text = Sam.values.tolist()
#Sam_Text

In [358]:
Sam_Words = []

for word in Sam_Text:
    if len(word)>=1:
        if word != '0':
            Sam_Words.append(word)
        
#Sam_Words 

In [359]:
Sam_Words_String = ' '.join(Sam_Words)
#Sam_Words_String

In [360]:
Sam_String_Length = len(Sam_Words_String)
Sam_String_Length

1130

In [361]:
Sam_Sentiment = TextBlob(Sam_Words_String).sentiment
Sam_Sentiment

Sentiment(polarity=-0.04237012987012988, subjectivity=0.5159752284752285)

### Sansa Stark

In [362]:
Sansa = season_five.loc['Sansa']
#print Sansa

In [363]:
Sansa_Text = Sansa.values.tolist()
#Sansa_Text

In [364]:
Sansa_Words = []

for word in Sansa_Text:
    if len(word)>=1:
        if word != '0':
            Sansa_Words.append(word)
        
#Sansa_Words 

In [365]:
Sansa_Words_String = ' '.join(Sansa_Words)
#Sansa_Words_String

In [366]:
Sansa_String_Length = len(Sansa_Words_String)
Sansa_String_Length

1045

In [367]:
Sansa_Sentiment = TextBlob(Sansa_Words_String).sentiment
Sansa_Sentiment

Sentiment(polarity=0.2036340852130326, subjectivity=0.6176691729323309)

### Shireen Baratheon

In [368]:
Shireen = season_five.loc['Shireen']
#print Shireen

In [369]:
Shireen_Text = Shireen.values.tolist()
#Shireen_Text

In [370]:
Shireen_Words = []

for word in Shireen_Text:
    if len(word)>=1:
        if word != '0':
            Shireen_Words.append(word)
        
#Shireen_Words 

In [371]:
Shireen_Words_String = ' '.join(Shireen_Words)
#Shireen_Words_String

In [372]:
Shireen_String_Length = len(Shireen_Words_String)
Shireen_String_Length

487

In [373]:
Shireen_Sentiment = TextBlob(Shireen_Words_String).sentiment
Shireen_Sentiment

Sentiment(polarity=0.4851190476190476, subjectivity=0.580952380952381)

### Stannis Baratheon¶

In [374]:
Stannis = season_five.loc['Stannis']
#print Stannis

In [375]:
Stannis_Text = Stannis.values.tolist()
#Stannis_Text

In [376]:
Stannis_Words = []

for word in Stannis_Text:
    if len(word)>=1:
        if word != '0':
            Stannis_Words.append(word)
        
#Stannis_Words 

In [377]:
Stannis_Words_String = ' '.join(Stannis_Words)
#Stannis_Words_String

In [378]:
Stannis_String_Length = len(Stannis_Words_String)
Stannis_String_Length

1437

In [379]:
Stannis_Sentiment = TextBlob(Stannis_Words_String).sentiment
Stannis_Sentiment

Sentiment(polarity=0.07684498041640897, subjectivity=0.5309334845049131)

### Theon Greyjoy

In [380]:
Theon = season_five.loc['Theon']
#print Theon

In [381]:
Theon_Text = Theon.values.tolist()
#Theon_Text

In [382]:
Theon_Words = []

for word in Theon_Text:
    if len(word)>=1:
        if word != '0':
            Theon_Words.append(word)
        
#Theon_Words 

In [383]:
Theon_Words_String = ' '.join(Theon_Words)
#Theon_Words_String

In [384]:
Theon_String_Length = len(Theon_Words_String)
Theon_String_Length

541

In [385]:
Theon_Sentiment = TextBlob(Theon_Words_String).sentiment
Theon_Sentiment

Sentiment(polarity=-0.1, subjectivity=0.4875)

### Tyrion Lannister

In [386]:
Tyrion = season_five.loc['Tyrion']
#print Tyrion

In [387]:
Tyrion_Text = Tyrion.values.tolist()
#Tyrion_Text

In [388]:
Tyrion_Words = []

for word in Tyrion_Text:
    if len(word)>=1:
        if word != '0':
            Tyrion_Words.append(word)
        
#Tyrion_Words 

In [389]:
Tyrion_Words_String = ' '.join(Tyrion_Words)
#Tyrion_Words_String

In [391]:
Tyrion_String_Length = len(Dany_Words_String)
Tyrion_String_Length

1839

In [392]:
Tyrion_Sentiment = TextBlob(Dany_Words_String).sentiment
Tyrion_Sentiment

Sentiment(polarity=0.1770833333333333, subjectivity=0.5620370370370371)

### Varys

In [393]:
Varys = season_five.loc['Varys']
#print Varys

In [394]:
Varys_Text = Varys.values.tolist()
#Varys_Text

In [395]:
Varys_Words = []

for word in Varys_Text:
    if len(word)>=1:
        if word != '0':
            Varys_Words.append(word)
        
#Varys_Words 

In [396]:
Varys_Words_String = ' '.join(Varys_Words)
#Varys_Words_String

In [397]:
Varys_String_Length = len(Varys_Words_String)
Varys_String_Length

892

In [398]:
Varys_Sentiment = TextBlob(Varys_Words_String).sentiment
Varys_Sentiment

Sentiment(polarity=0.3559818968909878, subjectivity=0.5848747212383576)